
# **FIAP – Fase 6 (Parte 2) – Comparação YOLO vs CNN**
**FarmTech Solutions – O Começo da Rede Neural**  
Autor: *preencha aqui* | RM: *preencha aqui*  
Data de geração deste notebook: 2025-10-14 03:58:35  

> **Objetivo:** Comparar três abordagens de visão computacional usando a mesma base da Entrega 1:  
> 1) **YOLOv5 Adaptável** (*pesos treinados na Entrega 1*)  
> 2) **YOLOv5 Padrão** (*treino baseline com hiperparâmetros default*)  
> 3) **CNN do zero (classificação)** (*a partir das imagens e rótulos da YOLO*)



## ✅ **Checklist de Pré-Requisitos**
- Sua base no **Google Drive** organizada conforme a Entrega 1 (imagens e rótulos YOLO-format).
- Um arquivo `dataset.yaml` no padrão YOLO com caminhos relativos para `train`, `val`, `test`, e a lista de `names` das classes.
- Os **pesos** treinados na Entrega 1 (por exemplo, `runs/train/expX/weights/best.pt`) salvos no Drive.
- Colab com GPU ativada (*Runtime > Change runtime type > GPU*).

> Se seu dataset tiver **apenas um objeto por imagem**, a conversão para classificação é direta. Caso haja múltiplos objetos por imagem, este notebook **seleciona a classe dominante** (maior número de anotações) para fins de classificação. Imagens com **empate** podem ser **ignoradas** para a CNN.



## 1) Ambiente: Montagem do Drive e Instalação de Dependências


In [ ]:

#@title Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive montado.")


In [ ]:

#@title Instalar YOLOv5 e dependências PyTorch/Ultralytics
!pip -q install --upgrade pip
!pip -q install ultralytics==8.2.103  # para utilidades e inferências
!git clone -q https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip -q install -r requirements.txt
%cd /content
print("✅ YOLOv5 instalado.")


In [ ]:

#@title Instalar dependências para a CNN (TensorFlow/Keras) e métricas
!pip -q install tensorflow==2.16.1 scikit-learn==1.5.2 matplotlib==3.8.4 pandas==2.2.2
print("✅ TensorFlow/Keras e libs instaladas.")



## 2) Variáveis de Caminho (Ajuste para seu Drive)
Preencha os caminhos abaixo antes de rodar o notebook.


In [ ]:

#@title 🔧 Configuração de caminhos (EDITE AQUI)
from pathlib import Path

# Caminho do YAML do dataset YOLO (no Drive)
DATASET_YAML_PATH = "/content/drive/MyDrive/fase6/dataset/dataset.yaml"  #@param {type:"string"}

# Caminho dos pesos treinados na Entrega 1 (YOLO adaptável)
CUSTOM_WEIGHTS_PATH = "/content/drive/MyDrive/fase6/yolo_runs/exp/weights/best.pt"  #@param {type:"string"}

# Pasta para salvar saídas desta Parte 2
OUTPUT_DIR = "/content/drive/MyDrive/fase6/entrega2_outputs"  #@param {type:"string"}

# Pasta temporária (local) para classificação (gerada a partir de rótulos YOLO)
CLASS_DATASET_DIR = "/content/classification_ds"

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print("✅ Config aplicado.")
print("DATASET_YAML_PATH:", DATASET_YAML_PATH)
print("CUSTOM_WEIGHTS_PATH:", CUSTOM_WEIGHTS_PATH)
print("OUTPUT_DIR:", OUTPUT_DIR)



## 3) YOLOv5 Padrão (Baseline) – Treino e Validação
Treinaremos a YOLOv5s com **hiperparâmetros padrão** (sem ajustes finos), para comparar com o modelo customizado da Entrega 1.


In [ ]:

#@title Treinar YOLOv5 (baseline - hiperparâmetros padrão)
import os, json, shutil, time, sys

%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 30 --data "$DATASET_YAML_PATH" --weights yolov5s.pt --name fase6_yolo_baseline --project /content/yolo_runs

# Registrar o caminho do experimento
BASELINE_EXP_DIR = "/content/yolo_runs/fase6_yolo_baseline"
print("✅ Treino baseline finalizado em:", BASELINE_EXP_DIR)
%cd /content


In [ ]:

#@title Avaliar YOLOv5 Baseline no conjunto de teste
%cd /content/yolov5
!python val.py --weights "/content/yolo_runs/fase6_yolo_baseline/weights/best.pt" --data "$DATASET_YAML_PATH" --task test --img 640 --name fase6_yolo_baseline_test --project /content/yolo_val
print("✅ Avaliação baseline concluída.")
%cd /content



## 4) YOLOv5 Adaptável (Pesos da Entrega 1) – Avaliação
Carregamos os **pesos customizados** (melhor epoch da Entrega 1) e avaliamos no conjunto de teste para comparação direta.


In [ ]:

#@title Avaliar YOLOv5 (pesos customizados da Entrega 1) no teste
from pathlib import Path

assert Path(CUSTOM_WEIGHTS_PATH).exists(), "❌ CUSTOM_WEIGHTS_PATH não encontrado."
%cd /content/yolov5
!python val.py --weights "$CUSTOM_WEIGHTS_PATH" --data "$DATASET_YAML_PATH" --task test --img 640 --name fase6_yolo_custom_test --project /content/yolo_val
print("✅ Avaliação YOLO customizada concluída.")
%cd /content



## 5) Conversão do Dataset YOLO → Classificação (2 classes)
A CNN requer pastas por classe (`train/classA`, `train/classB`, ...).  
Este passo usa os **arquivos de rótulo YOLO** para determinar a **classe dominante** da imagem e cria uma cópia em uma estrutura de diretórios de classificação.

> Regras:  
> - Se a imagem tiver **uma única classe**, usa-se aquela.  
> - Se tiver **várias classes**, escolhe-se a **mais frequente** na imagem.  
> - Se houver **empate**, a imagem é **ignoradas** para a CNN.


In [ ]:

#@title Conversão rótulos YOLO → dataset de classificação
import os, shutil, yaml, glob
from pathlib import Path
from collections import Counter

CLASS_DATASET_DIR = Path("/content/classification_ds")
if CLASS_DATASET_DIR.exists():
    shutil.rmtree(CLASS_DATASET_DIR)
CLASS_DATASET_DIR.mkdir(parents=True, exist_ok=True)

with open(DATASET_YAML_PATH, 'r') as f:
    ds = yaml.safe_load(f)

# ds['train'], ds['val'], ds['test'] podem ser pastas ou arquivos .txt com listas.
def resolve_image_paths(entry):
    p = Path(entry)
    if p.suffix.lower() == ".txt":
        with open(p) as fp:
            return [Path(line.strip()) for line in fp if line.strip()]
    else:
        # assume diretório com imagens
        exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp")
        paths = []
        for ext in exts:
            paths.extend(Path(p).rglob(ext))
        return paths

splits = {'train': ds['train'], 'val': ds.get('val', None), 'test': ds.get('test', None)}
names = ds.get('names', None)
if isinstance(names, dict):
    # YOLO às vezes usa dict {0:'A',1:'B'}
    class_names = [names[k] for k in sorted(names.keys(), key=int)]
else:
    class_names = list(names)

def yolo_label_path(img_path):
    # YOLO: images/ -> labels/, ext -> .txt
    img_path = Path(img_path)
    if "images" in img_path.parts:
        idx = img_path.parts.index("images")
        lbl_parts = list(img_path.parts)
        lbl_parts[idx] = "labels"
        lbl_path = Path(*lbl_parts).with_suffix(".txt")
        return lbl_path
    else:
        # fallback: assume pasta paralela labels com mesmo basename
        return img_path.with_suffix(".txt").parent.parent / "labels" / (img_path.stem + ".txt")

def dominant_class(label_file):
    if not label_file.exists():
        return None
    cls_ids = []
    with open(label_file) as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls_ids.append(int(parts[0]))
    if not cls_ids:
        return None
    count = Counter(cls_ids)
    most_common = count.most_common()
    if len(most_common) == 1 or (len(most_common) > 1 and most_common[0][1] > most_common[1][1]):
        return most_common[0][0]
    return None  # empate

def copy_for_split(split_name, sources):
    if not sources:
        return 0, 0
    kept, skipped = 0, 0
    for img in sources:
        lbl = yolo_label_path(img)
        cls_id = dominant_class(lbl)
        if cls_id is None:
            skipped += 1
            continue
        cls_name = class_names[cls_id] if class_names and cls_id < len(class_names) else f"class_{cls_id}"
        out_dir = CLASS_DATASET_DIR / split_name / cls_name
        out_dir.mkdir(parents=True, exist_ok=True)
        shutil.copy2(img, out_dir / img.name)
        kept += 1
    return kept, skipped

stats = {}
for split_name, entry in splits.items():
    if entry is None:
        continue
    imgs = resolve_image_paths(entry)
    kept, skipped = copy_for_split(split_name, imgs)
    stats[split_name] = dict(kept=kept, skipped=skipped)

print("✅ Conversão concluída.")
print(stats)
print("Estrutura criada em:", CLASS_DATASET_DIR)



## 6) CNN do Zero (Keras/TensorFlow) – Treino e Avaliação
Arquitetura simples: `Conv2D → MaxPool → Conv2D → MaxPool → Flatten → Dense → Dropout → Dense(softmax)`.


In [ ]:

#@title Preparar data loaders (ImageDataGenerator)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224  # resolução para a CNN
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    directory="/content/classification_ds/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    directory="/content/classification_ds/val",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_gen = test_datagen.flow_from_directory(
    directory="/content/classification_ds/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = train_gen.num_classes
print("Classes:", train_gen.class_indices)


In [ ]:

#@title Definir e treinar a CNN
from tensorflow.keras import layers, models

def build_cnn(input_shape=(224,224,3), num_classes=2):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = models.Model(inputs, outputs)
    return model

cnn = build_cnn(num_classes=NUM_CLASSES)
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
cnn.summary()

EPOCHS = 30  # baseline
history = cnn.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)

# Salvar pesos
cnn.save("/content/cnn_baseline.h5")
print("✅ CNN treinada e salva.")


In [ ]:

#@title Avaliar CNN no conjunto de teste (métricas de classificação)
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import pandas as pd
import time

# Predições
test_gen.reset()
t0 = time.time()
probs = cnn.predict(test_gen, verbose=0)
latency = (time.time() - t0) / max(1, test_gen.n)  # s/imagem
y_pred = np.argmax(probs, axis=1)
y_true = test_gen.classes
labels = list(test_gen.class_indices.keys())

acc = accuracy_score(y_true, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted", zero_division=0)
cm = confusion_matrix(y_true, y_pred)

print("Accuracy:", acc)
print("Precision (weighted):", prec)
print("Recall (weighted):", rec)
print("F1 (weighted):", f1)
print("Latency (s/img):", latency)

# Salvar relatório
report = classification_report(y_true, y_pred, target_names=labels, zero_division=0, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report.to_csv(f"{OUTPUT_DIR}/cnn_classification_report.csv", index=True)

# Plot Confusion Matrix
plt.figure(figsize=(4,4))
import itertools
plt.imshow(cm, interpolation='nearest')
plt.title("CNN - Matriz de Confusão")
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/cnn_confusion_matrix.png", bbox_inches='tight')
plt.show()

# Salvar métricas resumidas
summary = {
    "accuracy": float(acc),
    "precision_weighted": float(prec),
    "recall_weighted": float(rec),
    "f1_weighted": float(f1),
    "latency_s_per_image": float(latency)
}
import json, os
with open(f"{OUTPUT_DIR}/cnn_metrics.json", "w") as f:
    json.dump(summary, f, indent=2)
print("✅ Métricas CNN salvas em", OUTPUT_DIR)



## 7) Medir Latência de Inferência (YOLO vs CNN) e Consolidar Métricas
Executamos inferência na **pasta de teste** e registramos tempo médio por imagem.


In [ ]:

#@title Funções auxiliares para inferência YOLO em lote
import os, time, glob, json, shutil, yaml, pandas as pd
from pathlib import Path

def list_test_images_from_yaml(yaml_path):
    with open(yaml_path, 'r') as f:
        ds = yaml.safe_load(f)
    test_entry = ds.get('test')
    if not test_entry:
        raise ValueError("YAML não possui entrada 'test'.")
    p = Path(test_entry)
    if p.suffix.lower() == ".txt":
        with open(p) as fp:
            return [Path(line.strip()) for line in fp if line.strip()]
    else:
        exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp")
        paths = []
        for ext in exts:
            paths.extend(Path(p).rglob(ext))
        return paths

TEST_IMAGES = list_test_images_from_yaml(DATASET_YAML_PATH)
len(TEST_IMAGES), TEST_IMAGES[:3]


In [ ]:

#@title Inferência YOLOv5 baseline (tempo médio)
import time
%cd /content/yolov5
t0 = time.time()
!python detect.py --weights "/content/yolo_runs/fase6_yolo_baseline/weights/best.pt" --source "{' '.join(str(p) for p in TEST_IMAGES)}" --img 640 --save-txt --save-conf --exist-ok --project /content/yolo_detect --name baseline
t1 = time.time()
baseline_latency = (t1 - t0) / max(1, len(TEST_IMAGES))
print("Latency baseline (s/img):", baseline_latency)
%cd /content


In [ ]:

#@title Inferência YOLOv5 custom (tempo médio)
import time
%cd /content/yolov5
t0 = time.time()
!python detect.py --weights "$CUSTOM_WEIGHTS_PATH" --source "{' '.join(str(p) for p in TEST_IMAGES)}" --img 640 --save-txt --save-conf --exist-ok --project /content/yolo_detect --name custom
t1 = time.time()
custom_latency = (t1 - t0) / max(1, len(TEST_IMAGES))
print("Latency custom (s/img):", custom_latency)
%cd /content


In [ ]:

#@title Consolidar métricas (YOLO baseline, YOLO custom, CNN)
import json, pandas as pd, glob

# Ler métricas val/test da YOLO (mAP etc.) a partir dos resultados gerados por yolov5 (results.txt)
def read_yolo_results_txt(folder_glob):
    # Procura o arquivo results.txt mais recente nos diretórios correspondentes
    files = sorted(glob.glob(folder_glob, recursive=True), key=os.path.getmtime, reverse=True)
    metrics = {}
    for f in files:
        if os.path.basename(f) == "results.txt":
            with open(f) as fp:
                lines = fp.readlines()
            # Na última linha normalmente há "all" com mAP50, mAP50-95 etc., mas o formato pode variar por versão
            # Vamos guardar todo o arquivo como texto bruto também.
            metrics["raw"] = "".join(lines[-5:])
            break
    return metrics

baseline_metrics = read_yolo_results_txt("/content/yolo_val/fase6_yolo_baseline_test/**/results.txt")
custom_metrics   = read_yolo_results_txt("/content/yolo_val/fase6_yolo_custom_test/**/results.txt")

summary = {
    "yolo_baseline": {
        "val_text_tail": baseline_metrics.get("raw", ""),
        "latency_s_per_image": globals().get("baseline_latency", None)
    },
    "yolo_custom": {
        "val_text_tail": custom_metrics.get("raw", ""),
        "latency_s_per_image": globals().get("custom_latency", None)
    },
}

# Acrescentar também as métricas da CNN
with open(f"{OUTPUT_DIR}/cnn_metrics.json", "r") as f:
    cnn_m = json.load(f)
summary["cnn"] = cnn_m

# Salvar JSON consolidado
with open(f"{OUTPUT_DIR}/comparativo_metrics.json", "w") as f:
    json.dump(summary, f, indent=2)

import pandas as pd
df_summary = pd.DataFrame({
    "Modelo": ["YOLOv5 Baseline", "YOLOv5 Custom", "CNN (Keras)"],
    "Precisão/Notas": [
        "Ver tail do results.txt baseline (mAP)",
        "Ver tail do results.txt custom (mAP)",
        f"Acc={cnn_m.get('accuracy', None):.3f} | F1={cnn_m.get('f1_weighted', None):.3f}"
    ],
    "Latência (s/img)": [
        summary["yolo_baseline"]["latency_s_per_image"],
        summary["yolo_custom"]["latency_s_per_image"],
        summary["cnn"]["latency_s_per_image"]
    ]
})

df_summary.to_csv(f"{OUTPUT_DIR}/comparativo_metrics_table.csv", index=False)
print("✅ Tabela comparativa salva em CSV.")
df_summary



## 8) Discussão e Conclusões (preencha após execução)
- **Facilidade de uso/integração:**  
  - YOLOv5 baseline: *comente aqui.*  
  - YOLOv5 custom: *comente aqui.*  
  - CNN do zero: *comente aqui.*  

- **Precisão do modelo (mAP/Accuracy/F1):**  
  - YOLOv5 baseline: *X*  
  - YOLOv5 custom: *Y*  
  - CNN (acc/f1): *Z*  

- **Tempo de treinamento/customização:**  
  - YOLOv5 baseline: *X min*  
  - YOLOv5 custom: *Y min/épocas*  
  - CNN: *Z min*  

- **Tempo de inferência (s/imagem):**  
  - YOLOv5 baseline: *X*  
  - YOLOv5 custom: *Y*  
  - CNN: *Z*  

> **Conclusão:** Relacione os resultados com o **cenário de aplicação** (segurança patrimonial, controle de acessos, etc.). Normalmente a **YOLO custom** tende a oferecer melhor mAP e latência menor que uma CNN simples para tarefas de **detecção**. A CNN pode ser competitiva para **classificação** pura, com custo de implementação baixo, mas sem fornecer bounding boxes.



## Apêndice – Reprodutibilidade
- Configure seeds se desejar reprodutibilidade estrita (pode impactar performance):


In [ ]:

#@title (Opcional) Fixar seeds
import os, random, numpy as np, torch, tensorflow as tf
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
tf.random.set_seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
print("✅ Seeds configuradas (parcial).")



## Artefatos Gerados
- `yolo_runs/fase6_yolo_baseline/...` (treino YOLO baseline)  
- `yolo_val/fase6_yolo_baseline_test/...` (val YOLO baseline)  
- `yolo_val/fase6_yolo_custom_test/...` (val YOLO custom)  
- `classification_ds/` (dataset convertido para classificação)  
- `cnn_baseline.h5` (pesos CNN)  
- `comparativo_metrics.json`, `comparativo_metrics_table.csv`, `cnn_metrics.json`, `cnn_confusion_matrix.png` (em `OUTPUT_DIR`)  

Inclua prints das detecções em `/content/yolo_detect/` no seu **README** e **vídeo**.
